In [ ]:
img_data_cut = img_data[:,:,:,300]
img_cut = nib.Nifti1Image(img_cut, img.affine)
nib.save(img_cut,'img_cut1.nii')
cut = nib.load('img_cut.nii')

In [ ]:
"""Loading stim_site_mask"""
stim_mask_img = nib.load(stim_mask_path) #niftiimg
stim_mask_data = stim_mask_img.get_data() #numpy array
stim_mask_data.shape
stim_site_activity_vectors = apply_mask(img, stim_mask_img) #numpy array (600, 33)
stim_site_activity_vectors_avg = np.mean(stim_site_activity_vectors, axis=1)
stim_site_activity_vectors_avg.shape

In [ ]:
"""Plotting the stimulation site time series"""
import matplotlib.pyplot as plt
for time_series, label in zip(stim_site_time_series.T, stim_site_label):
    plt.plot(time_series, label=label)
plt.title('Stimulation Site Time Series')
plt.xlabel('Scan number')
plt.ylabel('Signal')
plt.legend()
plt.tight_layout()

In [ ]:
#uses mask_data to get masked voxels. Gets all Non-Zero voxels.
def get_masked_voxels(mask_data):
    masked_voxels = []
    for x in range(0, mask_data.shape[0]):
        for y in range(0, mask_data.shape[1]):
            for z in range(0, mask_data.shape[2]):
                if mask_data[x, y, z] != 0:
                    masked_voxels.append((x, y, z))
    return masked_voxels

In [ ]:
import math
global zero_voxel
zero_reference_mask = nib.load('/Users/hellwalker/Desktop/scans/reference_sites/zero_reference.nii.gz')
zero_reference_mask_data = zero_reference_mask.get_data()
zero_voxel = get_masked_voxels(zero_reference_mask_data)[0]

def mni2voxel(voxel):
    return (math.ceil(voxel[0]/2) + zero_voxel[0], math.ceil(voxel[1]/2) + zero_voxel[1], math.ceil(voxel[2]/2) 
            + zero_voxel[2])
def voxel2mni(voxel):
    return ((voxel[0] - zero_voxel[0])*2, (voxel[1] - zero_voxel[1])*2, (voxel[2] - zero_voxel[2])*2)

In [ ]:
thalamus_voxels = get_masked_voxels(thalamus_mask_data)
#thalamus_voxels_mni = [voxel2mni(voxel) for voxel in thalamus_voxels]
thalamus_voxels_mni = thalamus_voxels

In [ ]:
from nilearn import input_data
thalamus_masker = input_data.NiftiSpheresMasker(seeds=thalamus_voxels_mni, allow_overlap=True)
thalamus_time_series = thalamus_masker.fit_transform(img)
thalamus_time_series.shape

In [ ]:
labels = list(thalamus_voxels_mni)
from nilearn.connectome import ConnectivityMeasure
connectivity_measure = ConnectivityMeasure(kind='partial correlation')
partial_correlation_matrix = connectivity_measure.fit_transform(
    [thalamus_time_series])[0]
from nilearn import plotting

plotting.plot_connectome(partial_correlation_matrix, thalamus_voxels_mni,
                         title="Default Mode Network Connectivity")

# Display connectome with hemispheric projections.
# Notice (0, -52, 18) is included in both hemispheres since x == 0.
title = "Connectivity projected on hemispheres"
plotting.plot_connectome(partial_correlation_matrix, thalamus_voxels_mni, title=title,
                         display_mode='lyrz')

plotting.show()

In [ ]:
labels = list(thalamus_voxels_mni)
from nilearn.connectome import ConnectivityMeasure
connectivity_measure = ConnectivityMeasure(kind='partial correlation')
partial_correlation_matrix = connectivity_measure.fit_transform(
    [thalamus_activity_vectors])[0]
from nilearn import plotting

plotting.plot_connectome(partial_correlation_matrix, thalamus_voxels_mni,
                         title="Default Mode Network Connectivity")

# Display connectome with hemispheric projections.
# Notice (0, -52, 18) is included in both hemispheres since x == 0.
title = "Connectivity projected on hemispheres"
plotting.plot_connectome(partial_correlation_matrix, thalamus_voxels_mni, title=title,
                         display_mode='lyrz')

plotting.show()

In [ ]:
from nilearn import datasets

dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename, labels = dataset.maps, dataset.labels

print('Atlas ROIs are located in nifti image (4D) at: %s' %
      atlas_filename)  # 4D data

# One subject of resting-state data
data = datasets.fetch_adhd(n_subjects=1)
fmri_filenames = data.func[0]
print(dataset)

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', C=1.)
from sklearn import datasets
digits = datasets.load_digits()
data = digits.data
labels = digits.target
svc.fit(data[:-10], labels[:-10]) 
svc.predict(data[-10:])  

In [ ]:
"""
import numpy as np
correlation_matrix = np.corrcoef(time_series.T)
print correlation_matrix.shape

# Plot the correlation matrix
from matplotlib import pyplot as plt
plt.figure(figsize=(40, 40))
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)

plt.imshow(correlation_matrix, interpolation="nearest", cmap="RdBu_r",
           vmax=0.8, vmin=-0.8)

# Add labels and adjust margins
x_ticks = plt.xticks(range(len(labels) - 1), labels[1:], rotation=90)
y_ticks = plt.yticks(range(len(labels) - 1), labels[1:])
plt.gca().yaxis.tick_right()
plt.subplots_adjust(left=.01, bottom=.3, top=.99, right=.62)
"""